# Large Language Models

A number of Large Language Models (LLMs) are deployed in Dartmouth's compute infrastructure and available in `langchain_dartmouth`. 

LLMs generally come in two flavors:

- **Baseline completion models:** 
  - These models are trained to simply continue the given prompt by adding the next token.
- **Instruction-tuned chat models:** 
  - These models are built on baseline completion models, but further trained using a specific prompt format to allow a conversational back-and-forth.

Both kinds of models are supported by `langchain_dartmouth`, each using a separate component.

You can find all available models in the [AI API docs](https://ai.dartmouth.edu/openapi/index.html) under `Text Generation Inference`. Everything with `instruct` in its name is an instruction-tuned model, everything without it is a baseline completion model.

Let's explore both of these components! But before we we get started, we need to load our Dartmouth API key from the `.env` file:

In [ ]:
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

## Baseline Completion Models

Baseline completion models are trained to simply continue the given prompt by adding the next token. The continued prompt is then considered the next input to the model, which extends it by another token. This continues until a specified maximum number of tokens have been added, or until a special token called a _stop token_ is generated.

A popular use-case for completion models is to generate code. Let's try an example and have the LLM generate a function based on its signature!

All baseline completion models are available through the component `DartmouthLLM` in the submodule `langchain_dartmouth.llms`, so we first need to import that class:

In [ ]:
from langchain_dartmouth.llms import DartmouthLLM

We can now instantiate a specific LLM by specifying its name as it appears in the [AI API docs](https://ai.dartmouth.edu/openapi/index.html). Since the model will generate the continuation of our prompt, it usually makes sense to repeat our prompt in the response, which we can request by setting the parameter `return_full_text` to `True`:

In [ ]:
llm = DartmouthLLM(model_name="codellama-13b-python-hf", return_full_text=True)

We can now send a prompt to the model and receive its response by using the `invoke()` method:

In [ ]:
response = llm.invoke("def remove_digits(s: str) -> str:")
print(response)

Since they are only trained to continue the given prompt, completion models are not great at responding to chat-like prompts:

In [ ]:
response = llm.invoke("How can I define a class in Python?")
print(response)

As we can see, the model just continues the prompt in a way that is similar to what it has seen during its training. If we want to use it in a conversational way, we need to use an instruction-tuned chat model.

## Instruction-Tuned Chat Models

Instruction-tuned chat models are trained to follow a specific set of instructions that the model is expected to follow. These models can be used in conversational scenarios, where the user asks the model questions and the model replies with answers. The model will not just continue the prompt but also understand the context of the conversation preceding the prompt. To achieve this, baseline completion models are fine-tuned (i.e., further trained) on conversational text material that is formatted following a particular template. That is why we often see multiple variants of an LLM: the base model and the instruct version (see, e.g., [CodeLlama](https://huggingface.co/meta-llama/CodeLlama-13b-Instruct-hf)).

Let's see what happens if we ask an instruction-tuned model our question from the previous section:

In [ ]:
llm = DartmouthLLM(model_name="codellama-13b-instruct-hf")
response = llm.invoke("How can I define a class in Python?")
print(response)

Well, that does not seem very helpful... What went wrong here?

The problem is that the prompt we use during inference (when we invoke the model) needs to follow the same format that was used during the instruction-tuning. This format is not the same for every model! Let's try our prompt again using [CodeLlama's Instructions format](https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-code-llama/):

In [ ]:
response = llm.invoke("<s>[INST] How can I define a class in Python? [/INST] ")

print(response)

That looks a lot better! 

<div class="alert alert-block alert-info">

**Note:** You may notice that the last sentence gets cut off. This is due to the default value for the maximum number of generated tokens, which may be too low. You can set a higher limit when you instantiate the `DartmouthLLM` object. Check [the API reference](https://dartmouth-libraries.github.io/langchain-dartmouth/api.html#langchain_dartmouth.llms.DartmouthLLM) for more information.
</div>


Managing the prompt format can quickly get tedious, especially if you want to switch between different models. Fortunately, the `ChatDartmouth` component handles the prompt formatting "under-the-hood" and we can just pass the actual message when we invoke it:

In [ ]:
from langchain_dartmouth.llms import ChatDartmouth

llm = ChatDartmouth(model_name="llama-3-1-8b-instruct")
response = llm.invoke("How can I define a class in Python?")

print(response.content)

That looks a lot better! 

<div class="alert alert-block alert-info">

**Note:** `ChatDartmouth` returns more than just a raw string: It returns an `AIMessage` object, which you can learn more about in [LangChain's API reference](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.ai.AIMessage.html).

We will see more of these message objects in the recipe on [prompts](06-prompt-basics.ipynb)!
</div>




## Summary

In this recipe, we have learned how to use the `DartmouthLLM` and `ChatDartmouth` components. Which one to use depends on whether you are working with a baseline completion model or an instruction-tuned chat model:

Baseline completion models can only be used with `DartmouthLLM`. Instruction-tuned chat models should be used with `ChatDartmouth` so the correct prompt format is applied automatically.

You can also use `DartmouthLLM` with an instruction-tuned model, if you want full control over the exact string that is sent to the model. In that case, however, you might see unexpected responses if the prompt format is not correct.